In [12]:
from multiprocessing import Pool
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from time import sleep
from unicodedata import normalize

import pandas as pd
import numpy as np
import glob

from time import sleep
from random import randint

from datetime import datetime, timedelta

In [23]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_rows', 500)

In [37]:
def get_html(url):
    r = requests.get(url)
    r.encoding = 'utf-8'
#     r.encoding = r.apparent_encoding
    return r.text

def get_models_list(html):
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find('div', class_='ListingPopularMMM-module__items')
    items = divs.find_all('a', class_ = 'Link ListingPopularMMM-module__itemName')
    
    list_ = []
    
    for i in items:
        model = i.string
        href = i.get('href')
        
        keys = ['model', 'href']
        values = [model, href]
        
        dict_ = dict(zip(keys, values))
        list_.append(dict_)

    return(list_)

def get_pages_count(html):
    soup = BeautifulSoup(html, 'lxml')
#     pages = soup.find('div', class_ = 'ListingHead__title')
#     return pages
    try:
        divs = soup.find('span', class_ = 'ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages')
        pages = divs.find_all('a', class_ = 'Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page')[-1].get('href')
        total_pages = int(pages.split('=')[-1])
    except:
        total_pages = 1
    return total_pages


def get_pages_urls(html):
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find_all('h3', class_='ListingItemTitle-module__container ListingItemSequential-module__title')
    
    list_urls = []
    
    for d in divs:
        url = d.find('a', class_ = 'Link ListingItemTitle-module__link').get('href')
        if not 'new' in url:
            list_urls.append(url)

    return list_urls

def get_card_info(html):
    soup = BeautifulSoup(html, 'lxml')
#     divs = soup.find('div', class_ = 'sKvarnvCYkRQak6aJ_z4dQSltKajJ__container')
    divs = soup.find_all('div', class_='CardComplectation__group')
    
#     owner = soup.find('div', class_='ListingItem-module__main')
    items = soup.find('span', itemtype="http://schema.org/Car")
    ul = soup.find('ul', class_='CardInfo')
#     ul2 = 
    
    
    list_full = []
    
    for d in divs:
        item_name = d.find('span', class_='CardComplectation__itemName').text
        options = d.find_all('li', class_='CardComplectation__itemContentEl')
        
        list_options = []
        for l in options:
            list_options.append(l.text)
            
        keys = ['name', 'values']
        values = [item_name, list_options]
        
        dict_options = dict(zip(keys, values))
        list_full.append(dict_options)
      
    
    info = {'bodyType': items.find('meta', itemprop='bodyType').attrs['content'],
            'brand': items.find('meta', itemprop='brand').attrs['content'],
            'color': items.find('meta', itemprop='color').attrs['content'],
            'fuelType': items.find('meta', itemprop='fuelType').attrs['content'],
            'modelDate': items.find('meta', itemprop='modelDate').attrs['content'],
            'name': items.find('meta', itemprop='name').attrs['content'],
            'numberOfDoors': items.find('meta', itemprop='numberOfDoors').attrs['content'],
            'productionDate': items.find('meta', itemprop='productionDate').attrs['content'],
            'vehicleConfiguration': items.find('meta', itemprop='vehicleConfiguration').attrs['content'],
            'vehicleTransmission': items.find('meta', itemprop='vehicleTransmission').attrs['content'],
            'engineDisplacement': items.find('meta', itemprop='engineDisplacement').attrs['content'],
            'enginePower': items.find('meta', itemprop='enginePower').attrs['content'],
            'description': items.find('meta', itemprop='description').attrs['content'],
#             'kmAge': ul.find('li', class_='CardInfo__row CardInfo__row_kmAge').attrs['CardInfo__cell'],
            'kmAge': normalize('NFKD', ul.find('li', class_='CardInfo__row CardInfo__row_kmAge').find_all('span')[1].text) if ul.find('li', class_='CardInfo__row CardInfo__row_kmAge') else 0,
            'equipment' : list_full,
            'drive': ul.find('li', class_='CardInfo__row CardInfo__row_drive').find_all('span')[1].text,
            'wheel': ul.find('li', class_='CardInfo__row CardInfo__row_wheel').find_all('span')[1].text,
            'state': ul.find('li', class_='CardInfo__row CardInfo__row_state').find_all('span')[1].text,
            'ownersCount': ul.find('li', class_='CardInfo__row CardInfo__row_ownersCount').find_all('span')[1].text if ul.find('li', class_='CardInfo__row CardInfo__row_ownersCount') else np.nan,
            'pts': ul.find('li', class_='CardInfo__row CardInfo__row_pts').find_all('span')[1].text,
            'customs': ul.find('li', class_='CardInfo__row CardInfo__row_customs').find_all('span')[1].text,
            'owningTime': ul.find('li', class_='CardInfo__row CardInfo__row_owningTime').find_all('span')[1].text if ul.find('li', class_='CardInfo__row CardInfo__row_owningTime') else np.nan,
            'price': items.find('meta', itemprop='price').attrs['content'] if items.find('meta', itemprop='price').attrs['content'] else np.nan,
#             normalize
#             
            
#             
#             
           }
#     brand = divs.find('meta', itemprop='brand').attrs['content']
    return info


### Получаем ссылки по моделям.

In [4]:
html = get_html('https://auto.ru/moskva/cars/bmw/all/')

In [5]:
list_models = get_models_list(html)

In [6]:
list_models

[{'model': '1 серия', 'href': 'https://auto.ru/moskva/cars/bmw/1er/all/'},
 {'model': '3 серия', 'href': 'https://auto.ru/moskva/cars/bmw/3er/all/'},
 {'model': '5 серия', 'href': 'https://auto.ru/moskva/cars/bmw/5er/all/'},
 {'model': '6 серия', 'href': 'https://auto.ru/moskva/cars/bmw/6er/all/'},
 {'model': '7 серия', 'href': 'https://auto.ru/moskva/cars/bmw/7er/all/'},
 {'model': 'X1', 'href': 'https://auto.ru/moskva/cars/bmw/x1/all/'},
 {'model': 'X3', 'href': 'https://auto.ru/moskva/cars/bmw/x3/all/'},
 {'model': 'X4', 'href': 'https://auto.ru/moskva/cars/bmw/x4/all/'},
 {'model': 'X5', 'href': 'https://auto.ru/moskva/cars/bmw/x5/all/'},
 {'model': 'X6', 'href': 'https://auto.ru/moskva/cars/bmw/x6/all/'},
 {'model': 'X7', 'href': 'https://auto.ru/moskva/cars/bmw/x7/all/'}]

In [7]:
print(list_models[6:7])
# for x in list_models[3:4]:
#     model_name = x['model']
#     print(model_name)

[{'model': 'X3', 'href': 'https://auto.ru/moskva/cars/bmw/x3/all/'}]


In [42]:
list_models_manual = []

keys = ['model', 'href']
values = ['2 серия', 'https://auto.ru/moskva/cars/bmw/2er/all/']

dict_ = dict(zip(keys, values))
list_models_manual.append(dict_)

keys = ['model', 'href']
values = ['4 серия', 'https://auto.ru/moskva/cars/bmw/4/all/']

dict_ = dict(zip(keys, values))
list_models_manual.append(dict_)

keys = ['model', 'href']
values = ['8 серия', 'https://auto.ru/moskva/cars/bmw/8er/all/']

dict_ = dict(zip(keys, values))
list_models_manual.append(dict_)

keys = ['model', 'href']
values = ['M5', 'https://auto.ru/moskva/cars/bmw/m5/all/']

dict_ = dict(zip(keys, values))
list_models_manual.append(dict_)

keys = ['model', 'href']
values = ['X2', 'https://auto.ru/moskva/cars/bmw/x2/all/']

dict_ = dict(zip(keys, values))
list_models_manual.append(dict_)

keys = ['model', 'href']
values = ['X5M', 'https://auto.ru/moskva/cars/bmw/x5_m/all/']

dict_ = dict(zip(keys, values))
list_models_manual.append(dict_)

list_models_manual

[{'model': '2 серия', 'href': 'https://auto.ru/moskva/cars/bmw/2er/all/'},
 {'model': '4 серия', 'href': 'https://auto.ru/moskva/cars/bmw/4/all/'},
 {'model': '8 серия', 'href': 'https://auto.ru/moskva/cars/bmw/8er/all/'},
 {'model': 'M5', 'href': 'https://auto.ru/moskva/cars/bmw/m5/all/'},
 {'model': 'X2', 'href': 'https://auto.ru/moskva/cars/bmw/x2/all/'},
 {'model': 'X5M', 'href': 'https://auto.ru/moskva/cars/bmw/x5_m/all/'}]

In [43]:
# for x in list_models_test:
# for x in list_models[10:11]:
for x in list_models_manual:
    
    model_name = x['model']
    model_url = x['href']
    
    pages_count = get_pages_count(get_html(model_url))
#     pages_count = 1
    c = 0
    list_ad_urls_full = []
    
    print(model_name)
    
    for p in range(pages_count):
        page_number = p+1
        full_url = model_url + '?output_type=table&page=' + str(page_number)
        
#         print(full_url)

        list_ad_urls = get_pages_urls(get_html(full_url))
        
        print('processing page ' + str(page_number) + ' out of ' + str(pages_count))
#         sleep(randint(2,5))
    

        for url in list_ad_urls:
            r = get_html(url)
            print(url)
            info = get_card_info(r)

            if c == 0:
                df = pd.DataFrame([info])
            else:
                df = df.append([info], ignore_index=True)

            c += 1
            sleep(randint(2,10))

        list_ad_urls_full = list_ad_urls_full + list_ad_urls
        
    df['url'] = list_ad_urls_full
    df.to_csv(datetime.now().strftime("%Y%m%d") + '_BMW_auto_ru_used_' + model_name + '.csv', index=False)

    


2 серия
processing page 1 out of 2
https://auto.ru/cars/used/sale/bmw/2er/1099135322-2a6290a1/
https://auto.ru/cars/used/sale/bmw/2er/1098695372-6cc3b506/
https://auto.ru/cars/used/sale/bmw/2er/1098107738-310877d6/
https://auto.ru/cars/used/sale/bmw/2er/1096964514-128239f8/
https://auto.ru/cars/used/sale/bmw/2er/1096143574-4c89468f/
https://auto.ru/cars/used/sale/bmw/2er/1099020624-de6a5c7e/
https://auto.ru/cars/used/sale/bmw/2er/1093115912-19ec9176/
https://auto.ru/cars/used/sale/bmw/2er/1096933280-d09a656d/
https://auto.ru/cars/used/sale/bmw/2er/1094909690-6f52abe8/
https://auto.ru/cars/used/sale/bmw/2er/1093373920-fdc80a00/
https://auto.ru/cars/used/sale/bmw/2er/1098886458-55a0f059/
processing page 2 out of 2
4 серия
processing page 1 out of 3
https://auto.ru/cars/used/sale/bmw/4/1099632926-8f1ba633/
https://auto.ru/cars/used/sale/bmw/4/1095137106-c2447d63/
https://auto.ru/cars/used/sale/bmw/4/1099495716-fe6b48dd/
https://auto.ru/cars/used/sale/bmw/4/1096770360-7241fed8/
https://aut

https://auto.ru/cars/used/sale/bmw/x2/1099603580-24b9820f/
https://auto.ru/cars/used/sale/bmw/x2/1098386378-8e94c63c/
https://auto.ru/cars/used/sale/bmw/x2/1099177238-b7ae4dfc/
https://auto.ru/cars/used/sale/bmw/x2/1097651370-472a221e/
https://auto.ru/cars/used/sale/bmw/x2/1097737548-0128a570/
https://auto.ru/cars/used/sale/bmw/x2/1098453834-156b136a/
https://auto.ru/cars/used/sale/bmw/x2/1096713050-db9ca318/
https://auto.ru/cars/used/sale/bmw/x2/1099456084-f33329dc/
https://auto.ru/cars/used/sale/bmw/x2/1099456096-44de63cc/
https://auto.ru/cars/used/sale/bmw/x2/1098998540-a5151b9e/
https://auto.ru/cars/used/sale/bmw/x2/1099153244-9672b1f2/
processing page 2 out of 2
https://auto.ru/cars/used/sale/bmw/x2/1097320144-95f3b00b/
https://auto.ru/cars/used/sale/bmw/x2/1094417878-924e3c3c/
X5M
processing page 1 out of 2
https://auto.ru/cars/used/sale/bmw/x5_m/1093823514-a2e5e574/
https://auto.ru/cars/used/sale/bmw/x5_m/1097306208-126db979/
https://auto.ru/cars/used/sale/bmw/x5_m/1090101234-ec

In [17]:
glob.glob('data/parsed/*.csv')

['data/parsed\\20200721_BMW_auto_ru_used_1 серия.csv',
 'data/parsed\\20200721_BMW_auto_ru_used_2 серия Active Tourer.csv',
 'data/parsed\\20200721_BMW_auto_ru_used_2 серия Grand Tourer.csv',
 'data/parsed\\20200721_BMW_auto_ru_used_3 серия.csv',
 'data/parsed\\20200721_BMW_auto_ru_used_5 серия.csv',
 'data/parsed\\20200721_BMW_auto_ru_used_6 серия.csv',
 'data/parsed\\20200721_BMW_auto_ru_used_7 серия.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_2 серия.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_4 серия.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_8 серия.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_M5.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X1.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X2.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X3.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X4.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X5.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X5M.csv',
 'data/parsed\\20200722_BMW_auto_ru_used_X6.csv',
 'data/parsed\\2020072

In [18]:
df = pd.concat([pd.read_csv(f) for f in glob.glob('data/parsed/*.csv')], ignore_index = True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6342 entries, 0 to 6341
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              6342 non-null   object
 1   brand                 6342 non-null   object
 2   color                 6342 non-null   object
 3   fuelType              6342 non-null   object
 4   modelDate             6342 non-null   int64 
 5   name                  6342 non-null   object
 6   numberOfDoors         6342 non-null   int64 
 7   productionDate        6342 non-null   int64 
 8   vehicleConfiguration  6342 non-null   object
 9   vehicleTransmission   6342 non-null   object
 10  engineDisplacement    6342 non-null   object
 11  enginePower           6342 non-null   object
 12  description           6342 non-null   object
 13  kmAge                 6342 non-null   object
 14  equipment             6342 non-null   object
 15  drive                 6342 non-null   

In [32]:
# df
s = '75 925 км'
print(int(s.replace('км','').replace(' ','')))

75925


In [34]:
df['kmAge'] = df['kmAge'].apply(lambda x: int(x.replace('км','').replace(' ','')))

In [35]:
df

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,kmAge,equipment,drive,wheel,state,ownersCount,pts,customs,owningTime,price,url
0,хэтчбек 5 дв.,BMW,белый,бензин,2011,BMW 1 серия II (F20/F21) 116i,5,2012,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,1.6 LTR,136 N12,Автомобиль не требует ремонта. Комплект зимних...,75925,"[{'name': 'Элементы экстерьера', 'values': ['Л...",задний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,7 лет,800000,https://auto.ru/cars/used/sale/bmw/1er/1099563...
1,хэтчбек 5 дв.,BMW,чёрный,бензин,2015,BMW 1 серия II (F20/F21) Рестайлинг 118i,5,2015,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,1.6 LTR,136 N12,Машина эксплуатировалась бережно. Есть царапин...,91000,[],задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,850000,https://auto.ru/cars/used/sale/bmw/1er/1099604...
2,хэтчбек 3 дв.,BMW,серый,бензин,2007,BMW 1 серия I (E81/E82/E87/E88) Рестайлинг 118i,3,2008,HATCHBACK_3_DOORS AUTOMATIC 2.0,автоматическая,2.0 LTR,136 N12,Бмв покупалась в мае 2009 года в Москве\nПтс а...,100000,"[{'name': 'Безопасность', 'values': ['Система ...",задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,410000,https://auto.ru/cars/used/sale/bmw/1er/1099529...
3,хэтчбек 5 дв.,BMW,чёрный,бензин,2007,BMW 1 серия I (E81/E82/E87/E88) Рестайлинг 116i,5,2008,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,1.6 LTR,115 N12,Автомобиль в прекрасном состоянии для своих ле...,189000,"[{'name': 'Безопасность', 'values': ['Подушки ...",задний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,430000,https://auto.ru/cars/used/sale/bmw/1er/1099598...
4,хэтчбек 5 дв.,BMW,белый,бензин,2007,BMW 1 серия I (E81/E82/E87/E88) Рестайлинг 116i,5,2009,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,1.6 LTR,115 N12,Комплект зимних шин в подарок. Горит аварийная...,181000,"[{'name': 'Комфорт', 'values': ['Электростекло...",задний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,11 лет и 5 месяцев,400000,https://auto.ru/cars/used/sale/bmw/1er/1099604...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6337,внедорожник 5 дв.,BMW,серый,дизель,2018,BMW X7 I (G07) 30d,5,2020,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,249 N12,Автомобиль из демонстрационного парка по спец ...,7,"[{'name': 'Безопасность', 'values': ['Антипроб...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,6655200,https://auto.ru/cars/used/sale/bmw/x7/10986342...
6338,внедорожник 5 дв.,BMW,синий,дизель,2018,BMW X7 I (G07) 30d,5,2019,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,249 N12,Машина в отличном состоя установлена охранная...,33000,"[{'name': 'Салон', 'values': ['Тонированные ст...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,5978000,https://auto.ru/cars/used/sale/bmw/x7/10985567...
6339,внедорожник 5 дв.,BMW,чёрный,дизель,2018,BMW X7 I (G07) M50d,5,2019,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,400 N12,Цвет чёрный карбон. \n\nРучная сборка автомоби...,32000,"[{'name': 'Салон', 'values': ['Тонированные ст...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,8950000,https://auto.ru/cars/used/sale/bmw/x7/10990642...
6340,внедорожник 5 дв.,BMW,чёрный,дизель,2018,BMW X7 I (G07) 30d,5,2019,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,249 N12,Купить автомобиль с пробегом в РОЛЬФ — это быс...,7152,"[{'name': 'Безопасность', 'values': ['Подушка ...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,6299000,https://auto.ru/cars/used/sale/bmw/x7/10995342...


In [36]:
df.to_csv('train.csv', index=False)